In [15]:
import json
import re
from datetime import datetime
import random
import nltk
import pickle
from nameparser.parser import HumanName
from nltk.corpus import wordnet
import names
import gender_guesser.detector as gender

## Read in and clean text

In [16]:
with open('data/0.json', 'r') as f:
    data = json.load(f)
text = data['coreData']
#data = data['History']['text']
for i,p in enumerate(text):
    text[i] = re.sub("[\[].*?[\]]", "", p) # remove citations

text = ''.join(text)
print(text)

7 Year Bitch was an American punk rock band from Seattle, Washington. The band was active between 1990 and 1997 and released three albums over that time.The band formed at the same time as the emergence of the Riot Grrrl sub-genre, which is a sub-genre of punk music from the early to mid-1990s that emphasized the role of women in rock music. The Riot Grrrl movement began as a feminist response to the violence and misogyny that became more prominent in punk music in the mid to late 1980s, and 7 Year Bitch, an all female punk band, emerged as part of that sub-genre.
7 Year Bitch was formed in 1990 by vocalist Selene Vigil, guitarist Stefanie Sargent, bassist Elizabeth Davis and drummer Valerie Agnew.  Vigil, Sargent, and Agnew had been playing together in the Seattle band Barbie's Dream Car when their bassist left for Europe. They subsequently recruited Davis and renamed their band after the movie The Seven Year Itch, based on a suggestion by their friend, Ben London, of fellow Seattle b

## Replace Band Name

In [17]:
with open('titles', 'rb') as inf:
    titles = pickle.load(inf)

In [18]:
determiners = []
nouns = []
adjectives = []
for title in titles:
    tagged_title = nltk.pos_tag(nltk.word_tokenize(title.lower()))
    for tagged_word in tagged_title:
        word = tagged_word[0]
        pos = tagged_word[1]
        if pos == 'DT':
            determiners.append(word)
        elif pos == 'NN' or pos == 'NNS':
            nouns.append(word.capitalize())
        elif pos == 'JJ':
            adjectives.append(word.capitalize())

In [19]:
def getRandName():
    determiner = random.choice(determiners).capitalize()
    [adjective1, adjective2] = random.sample(adjectives, 2)
    [noun1, noun2] = random.sample(nouns, 2)

    title_format = random.randrange(3)
    if title_format == 0:
        return determiner + ' ' + adjective1 + ' ' + noun1
    elif title_format == 1:
        return determiner + ' ' + adjective1 + ' ' + adjective2 + ' ' + noun1
    elif title_format == 2:
        return determiner + ' ' + noun1 + ' and ' + determiner + ' ' + noun2

In [20]:
band_name = data['Name']
new_name = getRandName()
text = re.sub(band_name, new_name, text)
text

'The Béla and The Effigies was an American punk rock band from Seattle, Washington. The band was active between 1990 and 1997 and released three albums over that time.The band formed at the same time as the emergence of the Riot Grrrl sub-genre, which is a sub-genre of punk music from the early to mid-1990s that emphasized the role of women in rock music. The Riot Grrrl movement began as a feminist response to the violence and misogyny that became more prominent in punk music in the mid to late 1980s, and The Béla and The Effigies, an all female punk band, emerged as part of that sub-genre.\nThe Béla and The Effigies was formed in 1990 by vocalist Selene Vigil, guitarist Stefanie Sargent, bassist Elizabeth Davis and drummer Valerie Agnew.  Vigil, Sargent, and Agnew had been playing together in the Seattle band Barbie\'s Dream Car when their bassist left for Europe. They subsequently recruited Davis and renamed their band after the movie The Seven Year Itch, based on a suggestion by the

## Replace Years

In [21]:
def replace_years(s):
    year_lower_bound = -20
    year_upper_bound = datetime.now().year - max(list(map(int, re.findall("[0-9]{4}", text))))
    year_diff = random.randint(year_lower_bound, year_upper_bound)
    return re.sub("[0-9]{4}", lambda x: str(int(x.group()) + year_diff), text)

In [22]:
print(replace_years(text))

The Béla and The Effigies was an American punk rock band from Seattle, Washington. The band was active between 2001 and 2008 and released three albums over that time.The band formed at the same time as the emergence of the Riot Grrrl sub-genre, which is a sub-genre of punk music from the early to mid-2001s that emphasized the role of women in rock music. The Riot Grrrl movement began as a feminist response to the violence and misogyny that became more prominent in punk music in the mid to late 1991s, and The Béla and The Effigies, an all female punk band, emerged as part of that sub-genre.
The Béla and The Effigies was formed in 2001 by vocalist Selene Vigil, guitarist Stefanie Sargent, bassist Elizabeth Davis and drummer Valerie Agnew.  Vigil, Sargent, and Agnew had been playing together in the Seattle band Barbie's Dream Car when their bassist left for Europe. They subsequently recruited Davis and renamed their band after the movie The Seven Year Itch, based on a suggestion by their 

## Replace Names
https://stackoverflow.com/questions/20290870/improving-the-extraction-of-human-names-with-nltk
Shivansh bhandari's answer

In [23]:
person_list = []
person_names=person_list
def get_human_names(text):
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary = False)

    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        if len(person) > 1: #avoid grabbing lone surnames
            for part in person:
                name += part + ' '
            if name[:-1] not in person_list:
                person_list.append(name[:-1])
            name = ''
        person = []
#     print (person_list)

names_ = get_human_names(text)
for person in person_list:
    person_split = person.split(" ")
    for name in person_split:
        if wordnet.synsets(name):
            if(name in person):
                person_names.remove(person)
                break

print(person_names)

['Stefanie Sargent', 'Valerie Agnew', 'Ben London', "Sick 'Em", 'Roisin Dunne', 'Home Alive', 'Miami Beach']


TODO: function to replace names with autogen names
maybe create dict that matches old to new
also need to replace last names as well as first names (maybe first names too?)
i think can replace all full names, then can look for last or first because shouldn't be an issue that we already replaced some

In [24]:
name_genders = {}
d = gender.Detector()
for name in person_names:
    g = d.get_gender(name.split()[0])
    print(name, g)
    if g == 'male' or g == 'female':
        name_genders[name] = g
name_genders

Stefanie Sargent female
Valerie Agnew female
Ben London male
Sick 'Em unknown
Roisin Dunne female
Home Alive unknown
Miami Beach unknown


{'Stefanie Sargent': 'female',
 'Valerie Agnew': 'female',
 'Ben London': 'male',
 'Roisin Dunne': 'female'}

In [25]:
name_replacements = {}
for k,v in name_genders.items():
    name_replacements[k] = names.get_full_name(gender=v)
name_replacements

{'Stefanie Sargent': 'Danielle Grady',
 'Valerie Agnew': 'Lucy Nelson',
 'Ben London': 'David Gambino',
 'Roisin Dunne': 'Emily Dancy'}

In [26]:
for k,v in name_replacements.items():
    text = re.sub(k.split()[0], v.split()[0], text) # replace first name
    text = re.sub(k.split()[1], v.split()[1], text) # replace last name

In [27]:
print(text)

The Béla and The Effigies was an American punk rock band from Seattle, Washington. The band was active between 1990 and 1997 and released three albums over that time.The band formed at the same time as the emergence of the Riot Grrrl sub-genre, which is a sub-genre of punk music from the early to mid-1990s that emphasized the role of women in rock music. The Riot Grrrl movement began as a feminist response to the violence and misogyny that became more prominent in punk music in the mid to late 1980s, and The Béla and The Effigies, an all female punk band, emerged as part of that sub-genre.
The Béla and The Effigies was formed in 1990 by vocalist Selene Vigil, guitarist Danielle Grady, bassist Elizabeth Davis and drummer Lucy Nelson.  Vigil, Grady, and Nelson had been playing together in the Seattle band Barbie's Dream Car when their bassist left for Europe. They subsequently recruited Davis and renamed their band after the movie The Seven Year Itch, based on a suggestion by their frien

## Use info box for names?

In [29]:
with open('data/info-0.json', 'r') as inf:
    print(json.load(inf))

{'Origin': ['Seattle, Washington, USA'], 'Genres': ['Punk rock, riot grrrl, grunge'], 'Years active': ['1990â€“1997'], 'Labels': ["C/Z, Atlantic, Man's Ruin"], 'Associated acts': ["The Gits, Barbie's Dream Car, Alcohol Funnycar, Cistine, Von Iva, Clone"], 'Website': ['http://www.7yearbitchrocks.com'], 'Members': ['Selene VigilElizabeth DavisValerie Agnew Roisin Dunne'], 'Past members': ['Stefanie Sargent (Deceased)Lisa Faye Beatty (Deceased)']}
